In [1]:
import json

def load_json(fp):
    with open(fp) as jsf:
        return json.load(jsf)

In [2]:
import re

StartingTritets = {
  '000': 'ANNOTATED_T',
  '001': 'CESR_T_COUNT_CODE',
  '010': 'CESR_T_OP_CODE',
  '011': 'JSON',
  '100': 'MGPK', # fixMap
  '101': 'CBOR', # Map Major Type 5
  '111': 'CESR_B', # count code or op code
}

def get_file_length_in_bytes(file_path):
    """
    Reads a file in as bytes and returns its length.

    Parameters:
        file_path (str): The path to the file.

    Returns:
        int: The length of the file in bytes.
    """
    with open(file_path, 'rb') as file:
        file_bytes = file.read()
        return len(file_bytes)
def read_file_as_bytes(file_path):
    with open(file_path, 'rb') as file:
        file_bytes = file.read()
        return file_bytes

def is_bytes(obj):
    return isinstance(obj, bytes)
def is_string(obj):
    return isinstance(obj, str)

def byte_to_bits(byte):
    return format(byte, '08b')
  
def bytes_to_bits(byte_array):
    return ''.join(format(byte, '08b') for byte in byte_array)

def get_file_length_in_chars(file_path):
        """
        Reads a file as a string and returns its character length.
    
        Parameters:
            file_path (str): The path to the file.
    
        Returns:
            int: The length of the file in characters.
        """
        with open(file_path, 'r', encoding='utf-8') as file:
            file_content = file.read()
            return len(file_content)



def get_stream_tritet(stream):
      # to bytes
      if not is_bytes(stream):
          stream = stream.encode()
    

      first_byte = stream[0]
      first_bits = byte_to_bits(first_byte)
      first_tritet = first_bits[0:3]
    #   print("stream", stream)
    #   print("first_byte", first_byte)
    #   print("first_bits", first_bits)
    #   print('first_tritet', first_tritet)
    #   print(StartingTritets[first_tritet])
      print('first_tritet', first_tritet)
      return StartingTritets[first_tritet]
    
def get_version_string_info(v_string):
    # print(v_string)
    if not is_string(v_string):
        v_string = v_string.decode()
    
    v_string = v_string.replace('"', '')
    stop_delim = v_string.index('_')

    _protocol = v_string[0:4]
    _version = v_string[4:6]
    _kind = v_string[6:10]
    _size = v_string[10:stop_delim]
    _size_length =int(_size, 16)
    
    return {
      'protocol': _protocol,
      'version': _version,
      'kind': _kind, # serial
      '_size': _size, # digits
      'size': _size_length
    }

    # pattern1 = r'{"v":.*?_'
# pattern2 = r'-V..-'
# match = re.search(pattern1, c)

In [3]:
def print_messy_info(fp):
    byte_length = get_file_length_in_bytes(fp)
    
    f_bytes = read_file_as_bytes(fp)
    # f_str = f_
    print(f_bytes)
    print('bytes', byte_length)
    char_length = get_file_length_in_chars(fp)
    print('chars', char_length)
    stream_parser_type = get_stream_tritet(f_bytes)
    print('tritet kind', stream_parser_type)
    if stream_parser_type == 'JSON':
        version_string_pattern = r'{"v":.*?_"'
    match = re.search(version_string_pattern, f_bytes.decode())
    v_string = f_bytes[5:25].strip()
    # print(d['v'])
    # v_string = d['v']
    v_info = get_version_string_info(v_string)
    # v_info2 = get_version_string_info(d['v'])
    print(v_string)
    print(v_info)
    # print(v_info2)


In [4]:
fp = '../tests/acdcs/ecr-authorization-vlei-credential-ORIGINAL.json'
print_messy_info(fp)

b'{"v":"ACDC10JSON0004e4_","d":"EuF1gpodKbbqS0fqmUiOYf-MusuNvi0OmY8Js6SKSdfE","i":"EKXPX7hWw8KK5Y_Mxs2TOuCrGdN45vPIZ78NofRlVBws","ri":"EuqwB_iOD86eK0ynAhA6AYwWvPeBhvmbcmOD-9cCmiVU","s":"ELG17Q0M-uLZcjidzVbF7KBkoUhZa1ie3Az3Q_8aYi8s","a":{"d":"E9-86Jag34CrJpfNFz_-7E5HA0Dj0FvcYNoFVe7qwkiI","dt":"2022-08-25T14:07:30.536257+00:00","i":"EY4ldIBDZP4Tpnm3RX320BO0yz8Uz2nUSN-C409GnCJM","AID":"Esf8b_AngI1d0KbOFjPGIfpVani0HTagWeaYTLs14PlE","LEI":"6383001AJTYIGC8Y1X37","personLegalName":"John Smith","engagementContextRole":"Chief Executive Officer"},"e":{"d":"EsOf5_YgX_64z4YuHNFWLUnIKcyvsVQOe_vJ_638X6gE","le":{"n":"ESyLzoJC4L_1abXOEN4f6uNZCmhqyEHg2geBHFhJ8KDs","s":"ENPXp1vQzRF6JwIuS-mp2U8Uf1MoADoP_GqQ62VsDZWY"}},"r":{"d":"EDIai3Wkd-Z_4cezz9nYEcCK3KNH5saLvZoS_84JL6NU","usageDisclaimer":{"l":"Usage of a valid, unexpired, and non-revoked vLEI Credential, as defined in the associated Ecosystem Governance Framework, does not assert that the Legal Entity is trustworthy, honest, reputable in its business 

In [5]:
# file: ecr-authorization-vlei-credential.json 
## is output when kli saidify --file ~ecr-authorization-vlei-credential-ORIGINAL.json
## kli saidify recalculates the version string to the correct length (2122) and replaces the version string to represent this.
## this also changes the said to the value: "EDb9n2N2rDONME256eFcFYSTTn5qkKsu7u0DIOvi0rA3"
# kli version
# 1.1.19

fp2 = '../tests/acdcs/ecr-authorization-vlei-credential.json'
print_messy_info(fp2)

b'{"v": "ACDC10JSON00084a_", "d": "EDb9n2N2rDONME256eFcFYSTTn5qkKsu7u0DIOvi0rA3", "i": "EKXPX7hWw8KK5Y_Mxs2TOuCrGdN45vPIZ78NofRlVBws", "ri": "EuqwB_iOD86eK0ynAhA6AYwWvPeBhvmbcmOD-9cCmiVU", "s": "ELG17Q0M-uLZcjidzVbF7KBkoUhZa1ie3Az3Q_8aYi8s", "a": {"d": "E9-86Jag34CrJpfNFz_-7E5HA0Dj0FvcYNoFVe7qwkiI", "dt": "2022-08-25T14:07:30.536257+00:00", "i": "EY4ldIBDZP4Tpnm3RX320BO0yz8Uz2nUSN-C409GnCJM", "AID": "Esf8b_AngI1d0KbOFjPGIfpVani0HTagWeaYTLs14PlE", "LEI": "6383001AJTYIGC8Y1X37", "personLegalName": "John Smith", "engagementContextRole": "Chief Executive Officer"}, "e": {"d": "EsOf5_YgX_64z4YuHNFWLUnIKcyvsVQOe_vJ_638X6gE", "le": {"n": "ESyLzoJC4L_1abXOEN4f6uNZCmhqyEHg2geBHFhJ8KDs", "s": "ENPXp1vQzRF6JwIuS-mp2U8Uf1MoADoP_GqQ62VsDZWY"}}, "r": {"d": "EDIai3Wkd-Z_4cezz9nYEcCK3KNH5saLvZoS_84JL6NU", "usageDisclaimer": {"l": "Usage of a valid, unexpired, and non-revoked vLEI Credential, as defined in the associated Ecosystem Governance Framework, does not assert that the Legal Entity is trustwort

In [6]:
# see above -- original has been run through dict_to_said_str, which removes all white space between elements
## in the dictionary, then the sizes match.

In [7]:
def dict_to_said_str(data_dict):
    """
    Convert the dictionary to a JSON string 
    without extra spaces after commas and colons
    """
    json_str = json.dumps(data_dict, separators=(',', ':'))
    return json_str

In [8]:
fp2 = '../tests/acdcs/ecr-authorization-vlei-credential.json'
_dict = load_json(fp2)
dict_string = dict_to_said_str(_dict)

In [9]:
print(len(dict_string))
print(get_version_string_info(_dict['v']))

2122
{'protocol': 'ACDC', 'version': '10', 'kind': 'JSON', '_size': '00084a', 'size': 2122}


In [10]:
# simple-said/src$ python main.py ../tests/acdcs/ecr-authorization-vlei-credential.json d
# ****************************************************************************************
# ---------------- ../tests/acdcs/ecr-authorization-vlei-credential.json -----------------
# calc said:	 EDb9n2N2rDONME256eFcFYSTTn5qkKsu7u0DIOvi0rA3
# data said:	 EDb9n2N2rDONME256eFcFYSTTn5qkKsu7u0DIOvi0rA3
# match:		 True

